In [16]:
import pandas as pd
import numpy as np

In [30]:
read_data = pd.read_csv("data/cbc/train.csv", dtype='str')
#read_data = pd.read_csv("data/radio-canada/train.csv")
read_data.head()

,sessionId,contentId,eventId
0,user-000000,1.4782895,0
1,user-000000,1.4782714,1
2,user-000000,1.4782959,2
3,user-000001,1.4783578,0
4,user-000001,1.4783578,1


In [47]:
# Get all jsons of article data from REST API, store in hashmap: contentId -> json
import requests
import time
articleMap = {}
for row in read_data.itertuples(index=True):
    contentId = getattr(row, "contentId")
    try:
        if contentId not in articleMap:
            baseurl = 'https://www.cbc.ca/json/cmlink/'
            r = requests.get(url=baseurl+contentId)
            articleMap[contentId] = r.json()
    except:
        print(contentId)
        time.sleep(0.2)

1.4784021
1.4784021
1.4784021
1.4784021
1.4777577
1.4777843
1.4784021
1.4782904
1.4777843
1.4784021
1.4784021
1.4784021
1.4784021
1.4784021
1.4782904
1.4782904
1.4440614
1.4001179
1.4777843
1.4784021
1.4777843
1.4777843
1.4777843
1.4784021
1.4575179
1.2814407
1.4780513


KeyboardInterrupt: 

In [45]:
# create a new row for the headlines and body of article
import re
headlines = []
bodys = []
try:
    for row in read_data.itertuples(index=True):
        json = articleMap.get(getattr(row, "contentId"), {})
        headlines.append(json.get('headline',''))
        bodys.append(re.sub('<[^<]+?>', '', json.get('body', '')))
except:
    pass
read_data = read_data.assign(headline=pd.Series(headlines).values)
read_data = read_data.assign(body=pd.Series(bodys).values)

ValueError: Length of values does not match length of index

In [ ]:
read_data.head()
read_data.to_csv(r'data/clean/cbc/train.csv')

In [15]:
import csv
import requests

def readcsv(filename):
    with open(filename, newline='') as csvfile:
        return list(csv.reader(csvfile))
    
def parsecmlink(contentId, key):
    baseurl = 'https://www.cbc.ca/json/cmlink/'
    r = requests.get(url=baseurl+contentId)
    json = r.json()
    return json.get(key, '')

def augmentcsv(count=0):
    train = readcsv('data/cbc/train.csv')
    # reduce row count for debugging
    if count > 0:
        train = train[1:count+1]
    
    augmented = []
    for row in train:
        augmented.append(row + [parsecmlink(row[1])])
    
    return augmented

print(augmentcsv(30))

[['user-000000', '1.4782895', '0', 'Google tracks your movements, like it or not'], ['user-000000', '1.4782714', '1', "Despite lack of fans and competitors, Canada's NACAC success undeniable"], ['user-000000', '1.4782959', '2', 'Trump, Omarosa trade insults, charges in day-long spat'], ['user-000001', '1.4783578', '0', 'Federal task force on illegal border crossings to get new leader'], ['user-000001', '1.4783578', '1', 'Federal task force on illegal border crossings to get new leader'], ['user-000001', '1.4783578', '2', 'Federal task force on illegal border crossings to get new leader'], ['user-000002', '1.4779747', '0', 'Canadian residents hit by Trump tax dealt a new blow'], ['user-000003', '1.4782959', '0', 'Trump, Omarosa trade insults, charges in day-long spat'], ['user-000003', '1.4783329', '1', 'Ontario government wants statue of Sir John A. Macdonald that Victoria has put into storage'], ['user-000004', '1.4597112', '0', "Want to know everything Google knows about you? (Spoile

In [13]:
import requests
contentId='1.4784561'
baseurl = 'https://www.cbc.ca/json/cmlink/'
r = requests.get(url=baseurl+contentId)
r.json()['headline']

'B.C. Wildfires 2018: Smoky skies raise air quality concerns'